In [1]:
import numpy as np
from ultralytics import YOLO
import cv2
import cv2 as cv
import os
import matplotlib.pyplot as plt
#from programs.ResNet_Blocks_3D_four_blocks_test import resnet18
from programs.ResNet_Blocks_3D_four_blocks import resnet18
from programs.CustomDataset2 import CustomImageDataset
from programs.CustomDataset2 import padding
import torchvision.transforms as transforms
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
def bgsubList(folderName, filenames):

    frames = []
    for filename in filenames:
        frame = cv2.imread(folderName + '/' + filename)
        grayImage = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frames.append(grayImage)

    frameCount = len(frames)
    nSampFrame = min(np.fix(frameCount / 2), 100)

    # Creating a numpy array of the the sample frames
    firstSampFrame = True
    secondSampFrame = True
    for frameNum in np.fix(np.linspace(1, frameCount, int(nSampFrame))):
        if firstSampFrame:
            firstSampFrame = False
            sampFrames = frames[0]
            continue
        if secondSampFrame:
            secondSampFrame = False
            sampFrames = np.stack((sampFrames, frames[int((frameNum - 1))]), axis=0)
            continue
        sampFrames = np.vstack((sampFrames, np.array([frames[int((frameNum - 1))]])))

    sampFrames.sort(0)

    videobg = sampFrames[int(np.fix(nSampFrame * .9))]

    outputVid = []
    for frame in frames:
        # Subtract foreground from background image by allowing values beyond the 0 to 255 range of uint8
        difference_img = np.int16(videobg) - np.int16(frame)

        # Clip values in [0,255] range
        difference_img = np.clip(difference_img, 0, 255)

        # Convert difference image to uint8 for saving to video
        difference_img = np.uint8(difference_img)
        outputVid.append(difference_img)

    return outputVid

In [3]:
def isBoxFarFromEdge(box):
    edgeThreshold = 5
    imageSizeX, imageSizeY = 640, 640
    xDistance = np.min(imageSizeX - box[[0,2]])
    xDistance2 = np.min(box[[0,2]])
    xDistanceMin = min(xDistance, xDistance2)
    yDistance = np.min(imageSizeY - box[[1,3]])
    yDistance2 = np.min(box[[1,3]])
    yDistanceMin = min(yDistance, yDistance2)
    minDist = min(yDistanceMin, xDistanceMin)
    
    return minDist > edgeThreshold 

In [4]:
folder = '020116_012/'
files = os.listdir(folder)
files = [fileName for fileName in files if fileName.endswith('.bmp')]
files.sort()

bgsubIms = bgsubList(folder[:-1], files)

#im0 = cv.imread(folder + files[0])
im0 = bgsubIms[0]
print('bgsub done')

bgsub done


In [5]:
yoloWeigths = 'inputs/weights/orthographic_yolo/best.pt'

model = YOLO(yoloWeigths)

In [6]:
def drawBox(im, box):
    red = [0,0,255]
    box = np.copy(box)
    box = box.astype(int)
    sx, sy, bx, by = box
    im[sy,sx:bx + 1] = red
    im[by,sx:bx + 1] = red
    im[sy:by + 1, sx] = red
    im[sy:by + 1, bx] = red
    return im

In [7]:
imageSizeX, imageSizeY = 640, 640
fps = 500
#fps = cap.get( cv.CAP_PROP_FPS )
fourcc = cv.VideoWriter_fourcc('M','J','P','G')
#fourcc = cv.VideoWriter_fourcc(*'DIVX')
#outputPath = 'outputs/superimposed/' + videoName + '.avi'
outputPath = 'yoloVideo.avi'
out = cv.VideoWriter(outputPath, fourcc  , int( fps  ) ,(int(imageSizeX) , int( imageSizeY )))

#confidence_mask = result.boxes.conf.cpu().numpy() > .5
#boxList = result.boxes.xyxy.cpu().numpy()[confidence_mask]
#keypointsList = result.keypoints.xy.cpu().numpy()[confidence_mask]
#mask = [isBoxFarFromEdge(box) for box in boxList]
#boxList = boxList[mask]
#keypointsList = keypointsList[mask]
bar = tqdm(total = len(bgsubIms))

batchsize = 500
fc = 0
frames = []
amountOfFrames = len(bgsubIms)
while True:
    frame = bgsubIms[fc]
    fc += 1
    frame = np.stack((frame, frame, frame), axis = 2)
    frame = frame.astype(float)
    frame *= 255 / np.max(frame)
    frame = frame.astype(np.uint8)
    
    frames.append(frame)
    
    if len(frames) >= batchsize or fc >= amountOfFrames:
        results = model(frames, verbose = False)
        masterBoxList = []
        for result in results:
            confidence_mask = result.boxes.conf.cpu().numpy() > .5
            boxList = result.boxes.xyxy.cpu().numpy()[confidence_mask]
            mask = [isBoxFarFromEdge(box) for box in boxList]
            boxList = boxList[mask]
            masterBoxList.append(boxList)
        
        
        for frameIdx, _ in enumerate(frames):
            realFrameIdx = (fc - len(frames)) + frameIdx
            gray = bgsubIms[realFrameIdx]
            rgb = np.stack((gray,gray,gray), axis = 2)
            boxList = masterBoxList[frameIdx]
            for box in boxList:
                rgb = drawBox(rgb, box)
            
            out.write(rgb)
        bar.update(len(frames))
        frames = []
    if fc >= amountOfFrames: break
            




100%|██████████| 4977/4977 [01:40<00:00, 58.97it/s]

In [8]:
print(fc)
out.release()

4977


In [6]:
imageSizeX, imageSizeY = 640, 640

dataList = []
#confidence_mask = result.boxes.conf.cpu().numpy() > .5
#boxList = result.boxes.xyxy.cpu().numpy()[confidence_mask]
#keypointsList = result.keypoints.xy.cpu().numpy()[confidence_mask]
#mask = [isBoxFarFromEdge(box) for box in boxList]
#boxList = boxList[mask]
#keypointsList = keypointsList[mask]
bar = tqdm(total = len(bgsubIms))

batchsize = 500
fc = 0
frames = []
amountOfFrames = len(bgsubIms)
while True:
    frame = bgsubIms[fc]
    fc += 1
    frame = np.stack((frame, frame, frame), axis = 2)
    frame = frame.astype(float)
    frame *= 255 / np.max(frame)
    frame = frame.astype(np.uint8)
    
    frames.append(frame)
    
    if len(frames) >= batchsize or fc >= amountOfFrames:
        results = model(frames, verbose = False)
        masterBoxList = []
        for result in results:
            confidence_mask = result.boxes.conf.cpu().numpy() > .7
            classes = result.boxes.cls.cpu().numpy()[confidence_mask]
            boxList = result.boxes.xyxy.cpu().numpy()[confidence_mask]
            keypointsList = result.keypoints.xy.cpu().numpy()[confidence_mask]
            
            mask = [isBoxFarFromEdge(box) for box in boxList]
            boxList = boxList[mask]
            keypointsList = keypointsList[mask]
            classes = classes[mask]
            
            data = [classes, boxList, keypointsList]
            dataList.append(data)
            
        bar.update(len(frames))
        frames = []
    if fc >= amountOfFrames: break





100%|██████████| 4977/4977 [01:10<00:00, 109.09it/s]

In [7]:
import pickle
with open('orthographicVideoYoloData.pkl', 'wb') as b:
    pickle.dump(dataList, b)